In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import gzbuilder_analysis.parsing as pg
try:
    from gzbuilder_analysis.rendering.cuda import calculate_model
except ModuleNotFoundError:
    from gzbuilder_analysis.rendering import calculate_model
import gzbuilder_analysis.fitting as fg
import gzbuilder_analysis.config as cfg
from gzbuilder_analysis.fitting import Model, fit_model
import lib.galaxy_utilities as gu

In [15]:
expert_cls = gu.classifications.query('user_name == "tingard" and workflow_version >= 61.107')
assert expert_cls['subject_ids'].is_unique

In [16]:
fitting_metadata = pd.read_pickle('lib/fitting_metadata.pkl')
gal_angle_df = pd.read_csv('lib/gal-metadata.csv', index_col=0)

In [20]:
models = pd.Series([], name='models')
for _, cls in expert_cls.iterrows():
    subject_id = cls['subject_ids']
    if subject_id not in fitting_metadata.index:
        continue
    im = np.array(Image.open('lib/subject_data/{}/image.png'.format(subject_id)))[::-1]
    fm = fitting_metadata.loc[subject_id]
    data = fm['galaxy_data']
    sigma = fm['sigma_image']
    psf = fm['psf']
    gal = gal_angle_df.loc[subject_id]
    zoo_model = pg.parse_classification(
        cls,
        image_size=np.array(im.shape),
        size_diff=im.shape[0] / data.shape[0]
    )
    scaled_model = pg.scale_model(zoo_model, fm['size_diff'])
    model_dict = pg.reproject_model(
        scaled_model,
        wcs_in=fm['montage_wcs'],
        wcs_out=fm['original_wcs'],
    )
    models.loc[subject_id] = Model(
        model_dict,
        data,
        psf=psf,
        sigma_image=sigma
    )

In [26]:
models.to_pickle('lib/expert_models.pickle')
len(models)

41